In [3]:
import vk
import re
import time

#### >> получаем токен

In [4]:
session = vk.Session(access_token='PUT_ACCESS_TOKEN_HERE')
vkapi = vk.API(session, v='5.2')

#### Маленькие эксперименты

In [ ]:
vkapi.getServerTime()

In [4]:
profiles = vkapi.users.get(user_id=2)
profiles.append(vkapi.users.get(user_id=1))

In [5]:
#print(profiles[0:1]['last_name'])
print(len(profiles))

2


In [6]:
profiles = vkapi('users.get', user_id=1)
profiles[0]['last_name']

'Дуров'

In [7]:
vkapi.wall.post(message="test post")

{'post_id': 656}

In [8]:
s=vkapi.wall.get(domain='reylos',count=2) #get posts from the community

In [ ]:
print(s)

In [10]:
vkapi.users.get(user_id=1, fields='online, last_seen')

[{'first_name': 'Павел',
  'id': 1,
  'last_name': 'Дуров',
  'last_seen': {'platform': 1, 'time': 1527769493},
  'online': 0}]

#### По идее, должна сообщения отправлять, но что-то пошло не так

In [61]:
#vkapi.messages.send(users_id='0', messages='hi', peer_id='0')
#тут какая-то ошибочка

# request_params = {'messages': 'hi', 'oauth': '1', 'v': '5.73', 'users_id': '0', 'method': 'messages.send'}
# you should specify peer_id, user_id, domain, chat_id or user_ids param

### Возвращаемся к выгрузке сообщений

In [ ]:
friends = vkapi('friends.get') # получение всего списка друзей для пользователя
# friends = [1111111, 2222222, 33333333] # задаем друзей вручную 
print(friends)

In [ ]:
print(vkapi('users.get', user_id=25842353)[0]['last_name'])
print(vkapi('users.get', user_id=16123313)[0]['last_name'])
print(vkapi('users.get', user_id=21925237)[0]['last_name'])

In [14]:
#возвращает шапки диалогов
def get_dialogs(user_id):
    dialogs = vkapi('messages.getDialogs', user_id=user_id)
    return dialogs

In [ ]:
get_dialogs(0) #все шапки
get_dialogs(16123313) #шапка конкретного

#а КАК шапка диалога со МНОЙ? 0 или свой ид, ало
#а КАК не шапка а САМ диалог? history потому что, метод другой, лал

In [ ]:
vkapi('messages.getHistory', user_id=16123313, count=10, offset=0)

###### поправочка 1

In [17]:
SLEEP_TIME = 0.3

friend_dialog=get_dialogs(16123313)
time.sleep(SLEEP_TIME)
#dialog_len = friend_dialog[0]  так уже не надо

friend_dialog['count'] # вот так надо
#friend_dialog

12948

###### функции

In [203]:
SELF_ID = 1
SLEEP_TIME = 0.3

friends = vkapi('friends.get') # получение всего списка друзей для текущего пользователя

def get_dialogs(user_id):
    dialogs = vkapi('messages.getDialogs', user_id=user_id)
    return dialogs

def get_history(friends, sleep_time=0.3):
    all_history = []
    i = 0
    for friend in friends:
        friend_dialog = get_dialogs(friend)
        time.sleep(sleep_time)
        dialog_len = friend_dialog['count']           #здесь внесла поправку
        friend_history = []
        if dialog_len > 200:
            resid = dialog_len
            offset = 0
            while resid > 0:
                friend_history += vkapi('messages.getHistory', 
                    user_id=friend, 
                    count=200, 
                    offset=offset)['items']
                time.sleep(sleep_time)
                resid -= 200
                offset += 200
#                if resid > 0:
#                    print('--processing', friend, ':', resid, 
#                    'of', dialog_len, 'messages left')
            all_history += friend_history
        i +=1
#        print('processed', i, 'friends of', len(friends))
    return all_history

def get_messages_for_user(data, user_id):
    self_messages = []
    for dialog in data:
#        if type(dialog) == dict:
            if dialog['from_id'] == user_id: # удалила вот это dialog['uid'] == user_id and 
                m_text = re.sub("<br>", " ", dialog['body'])
                self_messages.append(m_text)
    print('Extracted', len(self_messages), 'messages in total')
    return self_messages

def save_to_file(data, file_name='output.txt'):
    with open(file_name, 'w', encoding='utf-8') as f:
        print(data, file=f)

In [200]:
all_history[0]['from_id']

237089588

In [202]:
if __name__ == '__main__':
    all_history = get_history(friends['items'], SLEEP_TIME) # здесь, опять же, уже friends['item'], а не просто friends
    save_to_file(all_history, 'raw.txt')

#    self_messages = get_messages_for_user(all_history, SELF_ID)
#    save_to_file(self_messages, 'sm_corpus.txt')
#ВСЕ РАВНО ЭТА ХУЙНЯ НЕ ХОЧЕТ РАБОТАТЬ, НО ОНА И НАХ НЕ СДАЛАСЬ В ОБЩЕМ

Extracted 0 messages in total


###### it's time to разбираться

In [ ]:
#get_history('237089588', sleep_time=0.3)

sleep_time=0.3
my_dialog = get_dialogs('237089588')
print('head of a dialog       ', my_dialog, '\r\n')
time.sleep(sleep_time)
dialog_len = my_dialog['count']
print('len of a dialog        ', dialog_len, '\r\n')
my_history = vkapi('messages.getHistory', 
                    user_id='237089588', 
                    count=200, 
                    offset=0)
my_history

###### ok, last 200 messages of my history we've got.
now let's get'em all

In [25]:
my_dialog = get_dialogs('sore_wa_himitsu_desu')
time.sleep(sleep_time)
dialog_len = my_dialog['count']           #здесь внесла поправку
my_history = []
#print(friend_history)
if dialog_len > 200:
    resid = dialog_len
    offset = 0
    while resid > 0:
        #print(my_history)
        my_history += vkapi('messages.getHistory', 
            user_id=237089588, 
            count=200, 
            offset=offset)['items'] #а вот и гвоздь. кот бы сомневался.
        #print(my_history)
        time.sleep(sleep_time)
        resid -= 200
        offset += 200

In [21]:
def save_to_file(data, file_name='output.txt'):
    with open(file_name, 'w', encoding='utf-8') as f:
        print(data, file=f)

In [26]:
#my_history
save_to_file(my_history, 'self_dialog.txt')